In [1]:
%reload_ext autoreload
%autoreload 2

In [12]:
import sys
if ".." not in sys.path:
    sys.path.append("..")  # Append pieye module root to sys.path

from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

from pieye.data import ClassificationDataset, get_filepaths

### Load Data

In [6]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
dataset = ClassificationDataset(annotations_filepath="../samples/annotations.txt", images_dir="../samples/images", transforms=data_transforms)

In [7]:
len(dataset)

11

In [8]:
dataset[2]

(tensor([[[ 94,  94,  94,  ..., 133, 133, 133],
          [ 95,  94,  94,  ..., 133, 133, 133],
          [ 95,  95,  95,  ..., 134, 134, 133],
          ...,
          [126, 128, 128,  ..., 103, 103, 103],
          [125, 125, 125,  ..., 104, 102, 102],
          [124, 124, 123,  ..., 103, 101, 100]],
 
         [[112, 112, 112,  ..., 168, 166, 166],
          [113, 112, 112,  ..., 166, 166, 166],
          [113, 113, 113,  ..., 168, 168, 166],
          ...,
          [115, 117, 117,  ...,  97,  97,  97],
          [117, 117, 117,  ...,  98,  99,  99],
          [116, 116, 115,  ...,  97,  98,  97]],
 
         [[ 98,  98,  96,  ..., 198, 201, 201],
          [ 99,  98,  96,  ..., 197, 199, 199],
          [ 99,  99,  97,  ..., 196, 196, 197],
          ...,
          [ 95,  97,  97,  ...,  81,  81,  81],
          [ 94,  94,  94,  ...,  82,  82,  82],
          [ 93,  93,  92,  ...,  81,  81,  80]]], dtype=torch.uint8),
 0)

In [ ]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True,  num_workers=4)
